In [252]:
import os
import json
import yaml
import pandas as pd
import numpy as np
import subprocess
import random

## Generating DB from existing JSON

In [2]:
# load existing meta data
with open('meta.json', 'r') as f:
    meta = json.load(f)

In [6]:
segment

{'labelID': 1,
 'SegmentDescription': 'Vertebrae T9',
 'SegmentAlgorithmType': 'AUTOMATIC',
 'SegmentAlgorithmName': 'TotalSegmentator',
 'SegmentedPropertyCategoryCodeSequence': {'CodeValue': '85756007',
  'CodingSchemeDesignator': 'SCT',
  'CodeMeaning': 'Tissue'},
 'SegmentedPropertyTypeCodeSequence': {'CodeValue': '272673000',
  'CodingSchemeDesignator': 'SCT',
  'CodeMeaning': 'Bone'},
 'recommendedDisplayRGBValue': [241, 214, 145]}

In [86]:
# collect some general data
categories = {}
types = {}
modifyers = {}

In [87]:
category_occurences = {}
type_ocurences = {}
modifyer_occurences = {}

In [89]:
modifyers

{'7771000': {'CodeValue': '7771000',
  'CodingSchemeDesignator': 'SCT',
  'CodeMeaning': 'Left'},
 '24028007': {'CodeValue': '24028007',
  'CodingSchemeDesignator': 'SCT',
  'CodeMeaning': 'Right'}}

In [90]:
segment

{'labelID': 1,
 'SegmentDescription': 'Vertebrae T9',
 'SegmentAlgorithmType': 'AUTOMATIC',
 'SegmentAlgorithmName': 'TotalSegmentator',
 'SegmentedPropertyCategoryCodeSequence': {'CodeValue': '85756007',
  'CodingSchemeDesignator': 'SCT',
  'CodeMeaning': 'Tissue'},
 'SegmentedPropertyTypeCodeSequence': {'CodeValue': '272673000',
  'CodingSchemeDesignator': 'SCT',
  'CodeMeaning': 'Bone'},
 'recommendedDisplayRGBValue': [241, 214, 145]}

In [88]:
# ietrate segments
for segment in meta['segmentAttributes']:

    # each segment contains just a single segment
    segment = segment[0]
    
    # desc
    desc = segment['SegmentDescription']
    
    #
    category = segment['SegmentedPropertyCategoryCodeSequence']
    category_code = category['CodeValue']
        
    if not category_code in categories:
        categories[category_code] = category
        category_occurences[category_code] = 1
    else:
        assert categories[category_code] == category
        category_occurences[category_code] += 1
    
    #
    typ = segment['SegmentedPropertyTypeCodeSequence']
    typ_code = typ['CodeValue']
    
    if not typ_code in types:
        types[typ_code] = typ
        type_ocurences[typ_code] = 1
    else:
        assert types[typ_code] == typ
        type_ocurences[typ_code] += 1
        
    #
    if not 'SegmentedPropertyTypeModifierCodeSequence' in segment:
        print('--> segment without mod', desc)
        continue
        
    mod = segment['SegmentedPropertyTypeModifierCodeSequence']
    mod_code = mod['CodeValue']
    
    if not mod_code in modifyers:
        modifyers[mod_code] = mod
        modifyer_occurences[mod_code] = 1
    else:
        if modifyers[mod_code] != mod:
            print(modifyers[mod_code])
            print(mod)
            
            print("-----")
        
        assert modifyers[mod_code] == mod
        modifyer_occurences[mod_code] += 1
    
    print(desc)
    

Left adrenal gland
Right adrenal gland
--> segment without mod Aorta
--> segment without mod Left autochthonous back muscle
--> segment without mod Right autochthonous back muscle
--> segment without mod Brain
Left clavicle
Right clavicle
--> segment without mod Colon
--> segment without mod Duodenum
--> segment without mod Esophagus
--> segment without mod Face
Left femur
Right femur
--> segment without mod Gallbladder
--> segment without mod Left gluteus maximus
--> segment without mod Right gluteus maximus
--> segment without mod Left gluteus medius
--> segment without mod Right gluteus medius
--> segment without mod Left gluteus minimus
--> segment without mod Right gluteus minimus
Left atrium
Right atrium
--> segment without mod Myocardium
--> segment without mod Left ventricle
--> segment without mod Right ventricle
--> segment without mod Left hip
--> segment without mod Right hip
Left humerus
Right humerus
Left iliac artery
Right iliac artery
Left iliac vein
Right iliac vein
--

In [14]:
typ_code

'23451007'

In [21]:
category_occurences

{}

In [91]:
categories_df = pd.DataFrame([c for _, c in categories.items()]).set_index('CodeValue')
types_df = pd.DataFrame([t for i, t in types.items()]).set_index('CodeValue')
modifyers_df = pd.DataFrame([m for _, m in modifyers.items()]).set_index('CodeValue')

In [92]:
categories_df.to_csv('categories.csv')
types_df.to_csv('types.csv')
modifyers_df.to_csv('modifyers.csv')

In [111]:
segids = []
segmentations = []
for seg in meta['segmentAttributes']:
    seg = seg[0]
    
    # extract dynamic parameters
    name = seg['SegmentDescription']
    segid = name.replace(' ', '_').upper()
    cat = seg['SegmentedPropertyCategoryCodeSequence']['CodeValue']
    typ = seg['SegmentedPropertyTypeCodeSequence']['CodeValue']
    mod = seg['SegmentedPropertyTypeModifierCodeSequence']['CodeValue'] if 'SegmentedPropertyTypeModifierCodeSequence' in seg else None
    color = ",".join([str(i) for i in seg['recommendedDisplayRGBValue']]) if 'recommendedDisplayRGBValue' in seg else None
    
    segids.append(segid)
    
    # create segmentation list
    segmentations.append({
        'id': segid,
        'name': name,
        'category': cat, 
        'type': typ, 
        'modifyer': mod,
        'color': color
    })
    
assert len(segids) == len(list(set(segids)))
segmentations_df = pd.DataFrame(segmentations).set_index('id')
segmentations_df.to_csv('segmentations.csv')

## Generate YML config from JSON meta

In [143]:
# get all output files of total segemntator (knowing that alphabetically sorted files match the segment order in json meta)
ts_file_names = sorted(f for f in os.listdir('/mnt/data1/IDC_NLST/seg/118760/CT-T1/LUNG/totalsegmentator') if f[-7:] == '.nii.gz' and f != 'heart_ventricle_left_combined.nii.gz')
with open('ts.output.txt', 'w') as f:
    f.write(",".join(ts_file_names))

In [144]:
ts_file_names = open('ts.output.txt').read().split(",")

In [152]:
# convert json to yaml
yaml_meta = {
    'dicomseg': {k: v for k, v in meta.items() if k != 'segmentAttributes'},
    'segments': {}
}

In [155]:
yaml_meta['dicomseg']['SegmentAlgorithmType'] = 'AUTOMATIC'
yaml_meta['dicomseg']['SegmentAlgorithmName'] = 'TotalSegmentator'

In [156]:
for i, file_name in enumerate(ts_file_names):
    seg = meta['segmentAttributes'][i][0]
    
    name = seg['SegmentDescription']
    segid = name.replace(' ', '_').upper()
    
    yaml_meta['segments'][segid] = file_name

In [157]:
# save yaml
with open('meta.yml', 'w') as f:
    yaml.dump(yaml_meta, f)

## Generate Meta JSON and DicomSeg from YML and DB

In [225]:
# generate meta and create dicomseg
with open('meta.yml', 'r') as f:
    yaml_meta = yaml.safe_load(f)

In [226]:
db = {
   'categories': pd.read_csv('categories.csv').set_index('CodeValue'),
   'types': pd.read_csv('types.csv').set_index('CodeValue'),
   'modifyers': pd.read_csv('modifyers.csv').set_index('CodeValue'),
   'segmentations': pd.read_csv('segmentations.csv').set_index('id')
}

In [295]:
def buildSegmentJsonBySegId(db, segid):
    
    seg = db['segmentations'].loc[segid]
        
    # mandatory
    json = {
        'labelID': 1,
        'SegmentDescription': seg['name'],
        'SegmentAlgorithmType': 'AUTOMATIC',
        'SegmentAlgorithmName': 'TotalSegmentator',
        'SegmentedPropertyCategoryCodeSequence': {'CodeValue': str(seg['category']), **dict(db['categories'].loc[seg['category']])},
        'SegmentedPropertyTypeCodeSequence': {'CodeValue': str(seg['type']), **dict(db['types'].loc[seg['type']])},
    }
    
    # optional
    if not pd.isnull(seg['modifyer']):
        json['SegmentedPropertyTypeModifierCodeSequence'] = {'CodeValue': str(seg['modifyer']), **dict(db['modifyers'].loc[seg['modifyer']])}
        
    if not pd.isnull(seg['color']):
        json['recommendedDisplayRGBValue'] = [int(c) for c in seg['color'].split(',')]
        
    # return
    return json

In [296]:
def generateJsonMeta(db, yaml_meta):
    
    # general properties / segment properties (keys)
    gpropk = ['ContentCreatorName', 'ClinicalTrialSeriesID', 'ClinicalTrialTimePointID', 'SeriesDescription', 'SeriesNumber', 'InstanceNumber', 'BodyPartExamined']
    spropk = ['SegmentAlgorithmType', 'SegmentAlgorithmName']

    # first copy the general properties
    json_meta = {k: yaml_meta['dicomseg'][k] for k in gpropk}
    json_meta['segmentAttributes'] = []

    # build segmentAttributes
    file_list = []
    for segid, seg_file in yaml_meta['segments'].items():
        json_meta['segmentAttributes'].append([{**buildSegmentJsonBySegId(db, segid), **{k: yaml_meta['dicomseg'][k] for k in spropk}}])
        file_list.append(seg_file)
        
    # return
    return json_meta, file_list

In [297]:
# test
json_meta, _ = generateJsonMeta(db, yaml_meta)
with open('meta2.json', 'w') as f:
    json.dump(json_meta, f)

In [298]:
# alternatively only process for those which exist on disc
def generateJsonMetaForFiles(db, yaml_meta, file_list):
    
    # general properties / segment properties (keys)
    gpropk = ['ContentCreatorName', 'ClinicalTrialSeriesID', 'ClinicalTrialTimePointID', 'SeriesDescription', 'SeriesNumber', 'InstanceNumber', 'BodyPartExamined']
    spropk = ['SegmentAlgorithmType', 'SegmentAlgorithmName']

    # first copy the general properties
    json_meta = {k: yaml_meta['dicomseg'][k] for k in gpropk}
    json_meta['segmentAttributes'] = []

    # build segmentAttributes
    for segid, seg_file in yaml_meta['segments'].items():
        if seg_file not in file_list:
            continue

        json_meta['segmentAttributes'].append([{**buildSegmentJsonBySegId(db, segid), **{k: yaml_meta['dicomseg'][k] for k in spropk}}])
        file_list.append(seg_file)
        
    if len(file_list) != len(json_meta['segmentAttributes']):
        print("WARNING: not all files were defined in yaml config.")
        
    # return
    return json_meta

In [299]:
def exportDicomSeg(yaml_file, dicom_dir, seg_dir, output_file, skip_empty_slices = False):
    tmp_json_file = 'temp-meta.json'

    # generate json meta
    json_meta, file_list = generateJsonMeta(db, yaml_meta)
    
    # build paths & assert they exist
    assert os.path.isdir(dicom_dir), "dicom dir not found"
    pred_segmasks_nifti_list = ",".join([os.path.join(seg_dir, seg_file) for seg_file in file_list])
    
    # create temporary meta file
    assert not os.path.isfile(tmp_json_file), "temp file already exists"
    with open(tmp_json_file, 'w') as f:
        json.dump(json_meta, f)
    
    # build command
    bash_command = list()
    bash_command += ["itkimage2segimage"]
    bash_command += ["--inputImageList", pred_segmasks_nifti_list]
    bash_command += ["--inputDICOMDirectory", dicom_dir]
    bash_command += ["--outputDICOM", output_file]
    bash_command += ["--inputMetadata", tmp_json_file]

    if skip_empty_slices == True:
        bash_command += ["--skip"]

    # run command
    bash_return = subprocess.run(bash_command, check = True, text = True)
    
    # remove temp file
    os.remove(tmp_json_file)
    
    # return subprocess
    return bash_return

In [300]:
yaml_file = 'meta.yml'
dicom_dir = '/mnt/data1/IDC_NLST/sorted/118760/CT-T1/LUNG/'
seg_dir = '/mnt/data1/IDC_NLST/seg/118760/CT-T1/LUNG/totalsegmentator'
output_file = 'ts-all.seg.dcm'

In [302]:
exportDicomSeg(yaml_file, dicom_dir, seg_dir, output_file)

dcmqi repository URL: git@github.com:QIICR/dcmqi.git revision: 1153738 tag: v1.2.5
Loaded segmentation from /mnt/data1/IDC_NLST/seg/118760/CT-T1/LUNG/totalsegmentator/aorta.nii.gz
Loaded segmentation from /mnt/data1/IDC_NLST/seg/118760/CT-T1/LUNG/totalsegmentator/brain.nii.gz
Loaded segmentation from /mnt/data1/IDC_NLST/seg/118760/CT-T1/LUNG/totalsegmentator/colon.nii.gz
Loaded segmentation from /mnt/data1/IDC_NLST/seg/118760/CT-T1/LUNG/totalsegmentator/duodenum.nii.gz
Loaded segmentation from /mnt/data1/IDC_NLST/seg/118760/CT-T1/LUNG/totalsegmentator/esophagus.nii.gz
Loaded segmentation from /mnt/data1/IDC_NLST/seg/118760/CT-T1/LUNG/totalsegmentator/face.nii.gz
Loaded segmentation from /mnt/data1/IDC_NLST/seg/118760/CT-T1/LUNG/totalsegmentator/gallbladder.nii.gz
Loaded segmentation from /mnt/data1/IDC_NLST/seg/118760/CT-T1/LUNG/totalsegmentator/inferior_vena_cava.nii.gz
Loaded segmentation from /mnt/data1/IDC_NLST/seg/118760/CT-T1/LUNG/totalsegmentator/adrenal_gland_left.nii.gz
Loaded

CompletedProcess(args=['itkimage2segimage', '--inputImageList', '/mnt/data1/IDC_NLST/seg/118760/CT-T1/LUNG/totalsegmentator/aorta.nii.gz,/mnt/data1/IDC_NLST/seg/118760/CT-T1/LUNG/totalsegmentator/brain.nii.gz,/mnt/data1/IDC_NLST/seg/118760/CT-T1/LUNG/totalsegmentator/colon.nii.gz,/mnt/data1/IDC_NLST/seg/118760/CT-T1/LUNG/totalsegmentator/duodenum.nii.gz,/mnt/data1/IDC_NLST/seg/118760/CT-T1/LUNG/totalsegmentator/esophagus.nii.gz,/mnt/data1/IDC_NLST/seg/118760/CT-T1/LUNG/totalsegmentator/face.nii.gz,/mnt/data1/IDC_NLST/seg/118760/CT-T1/LUNG/totalsegmentator/gallbladder.nii.gz,/mnt/data1/IDC_NLST/seg/118760/CT-T1/LUNG/totalsegmentator/inferior_vena_cava.nii.gz,/mnt/data1/IDC_NLST/seg/118760/CT-T1/LUNG/totalsegmentator/adrenal_gland_left.nii.gz,/mnt/data1/IDC_NLST/seg/118760/CT-T1/LUNG/totalsegmentator/heart_atrium_left.nii.gz,/mnt/data1/IDC_NLST/seg/118760/CT-T1/LUNG/totalsegmentator/autochthon_left.nii.gz,/mnt/data1/IDC_NLST/seg/118760/CT-T1/LUNG/totalsegmentator/clavicula_left.nii.gz,/m

itkimage2segimage --inputImageList {} --inputMetadata {} --inputDICOMDirectory {} --outputDICOM {}".

In [313]:
# one-line hex to rgb
h = '#affe'
[int(h.replace('#', '').ljust(6, '0')[i:i+2], 16) for i in (0, 2, 4)]

[175, 254, 0]